# Intro
placeholder

# Imports

In [32]:
# standard library imports
import csv
import datetime as dt
import json
import os
import statistics
import time

# third-party imports
import numpy as np
import pandas as pd
import requests

# customisations - ensure tables show all columns
pd.set_option("max_columns", 100)

# Get data
## 1) Create list of games
The API for SteamSpy (https://steamspy.com/api.php) can only return pages of 1000 results when the request is for 'all', so I need to loop the requests to get more than one page.  

In [33]:
def get_request(url, parameters=None):
    """Return json-formatted response of a get request using optional parameters.
    
    Parameters
    ----------
    url : string
    parameters : {'parameter': 'value'}
        parameters to pass as part of get request
    
    Returns
    -------
    json_data
        json-formatted response (dict-like)
    """
    try:
        response = requests.get(url=url, params=parameters)
    except SSLError as s:
        print('SSL Error:', s)
        
        for i in range(5, 0, -1):
            print('\rWaiting... ({})'.format(i), end='')
            time.sleep(1)
        print('\rRetrying.' + ' '*10)
        
        # recusively try again
        return get_request(url, parameters)
    
    if response:
        return response.json()
    else:
        # response is none usually means too many requests. Wait and try again 
        print('No response, waiting 10 seconds...')
        time.sleep(10)
        print('Retrying.')
        return get_request(url, parameters)

To get a mangable amount of useful information, I limit the requests to 35 pages, as anything over that has to little owners to be of use.  
Also, the columns returned by the call are not enough, so I limit the results to store the IDs and the names of the games.

In [34]:
def request_pages():
    pre_app_list=[0]*35
    #Due to the amount of games in the Steam store, I limit the call to the 35000 more ownded games
    for i in range(0,35):
        url = "https://steamspy.com/api.php?request=all&page="+str(i)
        json_data = get_request(url)
        steam_spy_all = pd.DataFrame.from_dict(json_data, orient='index')
        pre_app_list[i] = steam_spy_all[['appid', 'name']].sort_values('appid').reset_index(drop=True)
        
    
    app_list = pd.concat(pre_app_list).reset_index(drop=True)
    return app_list

In [35]:
# # Save the results only the first time
# game_list = request_pages()
# game_list.to_csv('data/game_list.csv', index=False)

# # instead read from stored csv
game_list = pd.read_csv('data/game_list.csv')

In [36]:
game_list

,appid,name
0,10,Counter-Strike
1,20,Team Fortress Classic
2,30,Day of Defeat
3,40,Deathmatch Classic
4,50,Half-Life: Opposing Force
...,...,...
34995,1812930,Santhai
34996,1814060,Virus at Home
34997,1816650,Nelson and the Magic Cauldron: The Journey
34998,1828900,Stickit


## 2) Download data from Steam
The Steam API returns data by ID, so I will loop through the `appid` column in `game_list` to get the dataframe.

In [37]:
def get_app_data(start, stop, parser, pause):
    """Return list of app data generated from parser.
    
    parser : function to handle request
    """
    app_data = []
    
    # iterate through each row of app_list, confined by start and stop
    for index, row in app_list[start:stop].iterrows():
        print('Current index: {}'.format(index), end='\r')
        
        appid = row['appid']
        name = row['name']

        # retrive app data for a row, handled by supplied parser, and append to list
        data = parser(appid, name)
        app_data.append(data)

        time.sleep(pause) # prevent overloading api with requests
    
    return app_data


def process_batches(parser, app_list, download_path, data_filename, index_filename,
                    columns, begin=0, end=-1, batchsize=100, pause=1):
    """Process app data in batches, writing directly to file.
    
    parser : custom function to format request
    app_list : dataframe of appid and name
    download_path : path to store data
    data_filename : filename to save app data
    index_filename : filename to store highest index written
    columns : column names for file
    
    Keyword arguments:
    
    begin : starting index (get from index_filename, default 0)
    end : index to finish (defaults to end of app_list)
    batchsize : number of apps to write in each batch (default 100)
    pause : time to wait after each api request (defualt 1)
    
    returns: none
    """
    print('Starting at index {}:\n'.format(begin))
    
    # by default, process all apps in app_list
    if end == -1:
        end = len(app_list) + 1
    
    # generate array of batch begin and end points
    batches = np.arange(begin, end, batchsize)
    batches = np.append(batches, end)
    
    apps_written = 0
    batch_times = []
    
    for i in range(len(batches) - 1):
        start_time = time.time()
        
        start = batches[i]
        stop = batches[i+1]
        
        app_data = get_app_data(start, stop, parser, pause)
        
        rel_path = os.path.join(download_path, data_filename)
        
        # writing app data to file
        with open(rel_path, 'a', newline='', encoding='utf-8') as f:
            writer = csv.DictWriter(f, fieldnames=columns, extrasaction='ignore')
            
            for j in range(3,0,-1):
                print("\rAbout to write data, don't stop script! ({})".format(j), end='')
                time.sleep(0.5)
            
            writer.writerows(app_data)
            print('\rExported lines {}-{} to {}.'.format(start, stop-1, data_filename), end=' ')
            
        apps_written += len(app_data)
        
        idx_path = os.path.join(download_path, index_filename)
        
        # writing last index to file
        with open(idx_path, 'w') as f:
            index = stop
            print(index, file=f)
            
        # logging time taken
        end_time = time.time()
        time_taken = end_time - start_time
        
        batch_times.append(time_taken)
        mean_time = statistics.mean(batch_times)
        
        est_remaining = (len(batches) - i - 2) * mean_time
        
        remaining_td = dt.timedelta(seconds=round(est_remaining))
        time_td = dt.timedelta(seconds=round(time_taken))
        mean_td = dt.timedelta(seconds=round(mean_time))
        
        print('Batch {} time: {} (avg: {}, remaining: {})'.format(i, time_td, mean_td, remaining_td))
            
    print('\nProcessing batches complete. {} apps written'.format(apps_written))

In [38]:
def reset_index(download_path, index_filename):
    """Reset index in file to 0."""
    rel_path = os.path.join(download_path, index_filename)
    
    with open(rel_path, 'w') as f:
        print(0, file=f)
        

def get_index(download_path, index_filename):
    """Retrieve index from file, returning 0 if file not found."""
    try:
        rel_path = os.path.join(download_path, index_filename)

        with open(rel_path, 'r') as f:
            index = int(f.readline())
    
    except FileNotFoundError:
        index = 0
        
    return index


def prepare_data_file(download_path, filename, index, columns):
    """Create file and write headers if index is 0."""
    if index == 0:
        rel_path = os.path.join(download_path, filename)

        with open(rel_path, 'w', newline='') as f:
            writer = csv.DictWriter(f, fieldnames=columns)
            writer.writeheader()

In [40]:
def parse_steam_request(appid, name):
    """Unique parser to handle data from Steam Store API.
    
    Returns : json formatted data (dict-like)
    """
    url = "http://store.steampowered.com/api/appdetails/"
    parameters = {"appids": appid}
    
    json_data = get_request(url, parameters=parameters)
    json_app_data = json_data[str(appid)]
    
    if json_app_data['success']:
        data = json_app_data['data']
    else:
        data = {'name': name, 'steam_appid': appid}
        
    return data


# Set file parameters
download_path = 'data'
steam_app_data = 'steam_app_data.csv'
steam_index = 'steam_index.txt'

steam_columns = [
    'type', 'name', 'steam_appid', 'required_age', 'is_free', 'controller_support',
    'dlc', 'detailed_description', 'about_the_game', 'short_description', 'fullgame',
    'supported_languages', 'header_image', 'website', 'pc_requirements', 'mac_requirements',
    'linux_requirements', 'legal_notice', 'drm_notice', 'ext_user_account_notice',
    'developers', 'publishers', 'demos', 'price_overview', 'packages', 'package_groups',
    'platforms', 'metacritic', 'reviews', 'categories', 'genres', 'screenshots',
    'movies', 'recommendations', 'achievements', 'release_date', 'support_info',
    'background', 'content_descriptors'
]

# Overwrites last index for demonstration (would usually store highest index so can continue across sessions)
reset_index(download_path, steam_index)

# Retrieve last index downloaded from file
index = get_index(download_path, steam_index)

# Wipe or create data file and write headers if index is 0
prepare_data_file(download_path, steam_app_data, index, steam_columns)

# Set end and chunksize for demonstration - remove to run through entire app list
process_batches(
    parser=parse_steam_request,
    app_list=game_list,
    download_path=download_path,
    data_filename=steam_app_data,
    index_filename=steam_index,
    columns=steam_columns,
    begin=index
)

# process_batches(
#     parser=parse_steam_request,
#     game_list=game_list,
#     download_path=download_path,
#     data_filename=steam_app_data,
#     index_filename=steam_index,
#     columns=steam_columns,
#     begin=index,
#     end=10,
#     batchsize=5
# )

Starting at index 0:

Exported lines 0-99 to steam_app_data.csv. Batch 0 time: 0:02:43 (avg: 0:02:43, remaining: 15:49:06)
Exported lines 100-199 to steam_app_data.csv. Batch 1 time: 0:02:42 (avg: 0:02:42, remaining: 15:44:16)
Exported lines 200-299 to steam_app_data.csv. Batch 2 time: 0:02:47 (avg: 0:02:44, remaining: 15:49:52)
Exported lines 300-399 to steam_app_data.csv. Batch 3 time: 0:02:39 (avg: 0:02:43, remaining: 15:39:56)
Exported lines 400-499 to steam_app_data.csv. Batch 4 time: 0:02:41 (avg: 0:02:42, remaining: 15:35:54)
Exported lines 500-599 to steam_app_data.csv. Batch 5 time: 0:02:54 (avg: 0:02:44, remaining: 15:44:36)
Exported lines 600-699 to steam_app_data.csv. Batch 6 time: 0:02:44 (avg: 0:02:44, remaining: 15:42:00)
Exported lines 700-799 to steam_app_data.csv. Batch 7 time: 0:02:45 (avg: 0:02:44, remaining: 15:39:49)
Exported lines 800-899 to steam_app_data.csv. Batch 8 time: 0:02:44 (avg: 0:02:44, remaining: 15:36:56)
Exported lines 900-999 to steam_app_data.csv.

Exported lines 7600-7699 to steam_app_data.csv. Batch 76 time: 0:02:38 (avg: 0:02:49, remaining: 12:49:40)
Exported lines 7700-7799 to steam_app_data.csv. Batch 77 time: 0:02:39 (avg: 0:02:48, remaining: 12:46:19)
Exported lines 7800-7899 to steam_app_data.csv. Batch 78 time: 0:02:37 (avg: 0:02:48, remaining: 12:42:51)
Exported lines 7900-7999 to steam_app_data.csv. Batch 79 time: 0:02:38 (avg: 0:02:48, remaining: 12:39:28)
Exported lines 8000-8099 to steam_app_data.csv. Batch 80 time: 0:02:39 (avg: 0:02:48, remaining: 12:36:10)
Exported lines 8100-8199 to steam_app_data.csv. Batch 81 time: 0:02:39 (avg: 0:02:48, remaining: 12:32:51)
Exported lines 8200-8299 to steam_app_data.csv. Batch 82 time: 0:02:39 (avg: 0:02:48, remaining: 12:29:34)
Exported lines 8300-8399 to steam_app_data.csv. Batch 83 time: 0:02:38 (avg: 0:02:48, remaining: 12:26:15)
Exported lines 8400-8499 to steam_app_data.csv. Batch 84 time: 0:02:39 (avg: 0:02:48, remaining: 12:22:58)
Exported lines 8500-8599 to steam_app

Exported lines 15100-15199 to steam_app_data.csv. Batch 151 time: 0:02:43 (avg: 0:02:45, remaining: 9:06:32)
Exported lines 15200-15299 to steam_app_data.csv. Batch 152 time: 0:02:42 (avg: 0:02:45, remaining: 9:03:44)
Exported lines 15300-15399 to steam_app_data.csv. Batch 153 time: 0:02:42 (avg: 0:02:45, remaining: 9:00:56)
Exported lines 15400-15499 to steam_app_data.csv. Batch 154 time: 0:02:43 (avg: 0:02:45, remaining: 8:58:08)
Exported lines 15500-15599 to steam_app_data.csv. Batch 155 time: 0:02:40 (avg: 0:02:45, remaining: 8:55:18)
Exported lines 15600-15699 to steam_app_data.csv. Batch 156 time: 0:02:44 (avg: 0:02:45, remaining: 8:52:33)
Exported lines 15700-15799 to steam_app_data.csv. Batch 157 time: 0:02:40 (avg: 0:02:45, remaining: 8:49:43)
Exported lines 15800-15899 to steam_app_data.csv. Batch 158 time: 0:02:43 (avg: 0:02:45, remaining: 8:46:56)
Exported lines 15900-15999 to steam_app_data.csv. Batch 159 time: 0:02:44 (avg: 0:02:45, remaining: 8:44:11)
Exported lines 1600

Exported lines 22600-22699 to steam_app_data.csv. Batch 226 time: 0:02:42 (avg: 0:02:43, remaining: 5:36:07)
Exported lines 22700-22799 to steam_app_data.csv. Batch 227 time: 0:02:36 (avg: 0:02:43, remaining: 5:33:20)
Exported lines 22800-22899 to steam_app_data.csv. Batch 228 time: 0:02:37 (avg: 0:02:43, remaining: 5:30:35)
Exported lines 22900-22999 to steam_app_data.csv. Batch 229 time: 0:02:37 (avg: 0:02:43, remaining: 5:27:49)
Exported lines 23000-23099 to steam_app_data.csv. Batch 230 time: 0:02:41 (avg: 0:02:43, remaining: 5:25:06)
Exported lines 23100-23199 to steam_app_data.csv. Batch 231 time: 0:02:37 (avg: 0:02:43, remaining: 5:22:21)
Exported lines 23200-23299 to steam_app_data.csv. Batch 232 time: 0:02:38 (avg: 0:02:43, remaining: 5:19:36)
Exported lines 23300-23399 to steam_app_data.csv. Batch 233 time: 0:02:36 (avg: 0:02:42, remaining: 5:16:50)
Exported lines 23400-23499 to steam_app_data.csv. Batch 234 time: 0:02:37 (avg: 0:02:42, remaining: 5:14:05)
Exported lines 2350

Exported lines 30100-30199 to steam_app_data.csv. Batch 301 time: 0:02:44 (avg: 0:02:42, remaining: 2:12:20)
Exported lines 30200-30299 to steam_app_data.csv. Batch 302 time: 0:02:38 (avg: 0:02:42, remaining: 2:09:38)
Exported lines 30300-30399 to steam_app_data.csv. Batch 303 time: 0:02:41 (avg: 0:02:42, remaining: 2:06:55)
Exported lines 30400-30499 to steam_app_data.csv. Batch 304 time: 0:02:40 (avg: 0:02:42, remaining: 2:04:13)
Exported lines 30500-30599 to steam_app_data.csv. Batch 305 time: 0:02:41 (avg: 0:02:42, remaining: 2:01:31)
Exported lines 30600-30699 to steam_app_data.csv. Batch 306 time: 0:02:39 (avg: 0:02:42, remaining: 1:58:48)
Exported lines 30700-30799 to steam_app_data.csv. Batch 307 time: 0:02:40 (avg: 0:02:42, remaining: 1:56:06)
Exported lines 30800-30899 to steam_app_data.csv. Batch 308 time: 0:02:40 (avg: 0:02:42, remaining: 1:53:24)
Exported lines 30900-30999 to steam_app_data.csv. Batch 309 time: 0:02:45 (avg: 0:02:42, remaining: 1:50:42)
Exported lines 3100

## 3) Download data from SteamSpy

In [30]:
def parse_steamspy_request(appid, name):
    """Parser to handle SteamSpy API data."""
    url = "https://steamspy.com/api.php"
    parameters = {"request": "appdetails", "appid": appid}
    
    json_data = get_request(url, parameters)
    return json_data


# set files and columns
download_path = 'data'
steamspy_data = 'steamspy_data.csv'
steamspy_index = 'steamspy_index.txt'

steamspy_columns = [
    'appid', 'name', 'developer', 'publisher', 'score_rank', 'positive',
    'negative', 'userscore', 'owners', 'average_forever', 'average_2weeks',
    'median_forever', 'median_2weeks', 'price', 'initialprice', 'discount',
    'languages', 'genre', 'ccu', 'tags'
]

reset_index(download_path, steamspy_index)
index = get_index(download_path, steamspy_index)

# Wipe data file if index is 0
prepare_data_file(download_path, steamspy_data, index, steamspy_columns)

process_batches(
    parser=parse_steamspy_request,
    app_list=app_list,
    download_path=download_path, 
    data_filename=steamspy_data,
    index_filename=steamspy_index,
    columns=steamspy_columns,
    begin=index,
    pause=0.3
)

# process_batches(
#     parser=parse_steamspy_request,
#     app_list=app_list,
#     download_path=download_path, 
#     data_filename=steamspy_data,
#     index_filename=steamspy_index,
#     columns=steamspy_columns,
#     begin=index,
#     end=20,
#     batchsize=5,
#     pause=0.3
# )

Starting at index 0:

Exported lines 0-99 to steamspy_data.csv. Batch 0 time: 0:00:49 (avg: 0:00:49, remaining: 4:44:30)
Exported lines 100-199 to steamspy_data.csv. Batch 1 time: 0:00:49 (avg: 0:00:49, remaining: 4:43:06)
Exported lines 200-299 to steamspy_data.csv. Batch 2 time: 0:00:50 (avg: 0:00:49, remaining: 4:44:35)
Exported lines 300-399 to steamspy_data.csv. Batch 3 time: 0:00:49 (avg: 0:00:49, remaining: 4:43:44)
Exported lines 400-499 to steamspy_data.csv. Batch 4 time: 0:00:49 (avg: 0:00:49, remaining: 4:42:23)
Exported lines 500-599 to steamspy_data.csv. Batch 5 time: 0:00:49 (avg: 0:00:49, remaining: 4:41:54)
Exported lines 600-699 to steamspy_data.csv. Batch 6 time: 0:00:49 (avg: 0:00:49, remaining: 4:41:07)
Exported lines 700-799 to steamspy_data.csv. Batch 7 time: 0:00:48 (avg: 0:00:49, remaining: 4:39:51)
Exported lines 800-899 to steamspy_data.csv. Batch 8 time: 0:00:51 (avg: 0:00:49, remaining: 4:40:08)
Exported lines 900-999 to steamspy_data.csv. Batch 9 time: 0:00

Exported lines 7800-7899 to steamspy_data.csv. Batch 78 time: 0:00:51 (avg: 0:00:50, remaining: 3:44:47)
Exported lines 7900-7999 to steamspy_data.csv. Batch 79 time: 0:00:52 (avg: 0:00:50, remaining: 3:44:05)
Exported lines 8000-8099 to steamspy_data.csv. Batch 80 time: 0:00:52 (avg: 0:00:50, remaining: 3:43:22)
Exported lines 8100-8199 to steamspy_data.csv. Batch 81 time: 0:00:50 (avg: 0:00:50, remaining: 3:42:34)
Exported lines 8200-8299 to steamspy_data.csv. Batch 82 time: 0:00:50 (avg: 0:00:50, remaining: 3:41:44)
Exported lines 8300-8399 to steamspy_data.csv. Batch 83 time: 0:00:49 (avg: 0:00:50, remaining: 3:40:53)
Exported lines 8400-8499 to steamspy_data.csv. Batch 84 time: 0:00:49 (avg: 0:00:50, remaining: 3:40:02)
Exported lines 8500-8599 to steamspy_data.csv. Batch 85 time: 0:00:49 (avg: 0:00:50, remaining: 3:39:11)
Exported lines 8600-8699 to steamspy_data.csv. Batch 86 time: 0:00:49 (avg: 0:00:50, remaining: 3:38:21)
Exported lines 8700-8799 to steamspy_data.csv. Batch 87

Exported lines 15500-15599 to steamspy_data.csv. Batch 155 time: 0:00:46 (avg: 0:00:49, remaining: 2:38:25)
Exported lines 15600-15699 to steamspy_data.csv. Batch 156 time: 0:00:46 (avg: 0:00:49, remaining: 2:37:33)
Exported lines 15700-15799 to steamspy_data.csv. Batch 157 time: 0:00:46 (avg: 0:00:49, remaining: 2:36:40)
Exported lines 15800-15899 to steamspy_data.csv. Batch 158 time: 0:00:46 (avg: 0:00:49, remaining: 2:35:48)
Exported lines 15900-15999 to steamspy_data.csv. Batch 159 time: 0:00:47 (avg: 0:00:49, remaining: 2:34:57)
Exported lines 16000-16099 to steamspy_data.csv. Batch 160 time: 0:00:46 (avg: 0:00:49, remaining: 2:34:05)
Exported lines 16100-16199 to steamspy_data.csv. Batch 161 time: 0:00:46 (avg: 0:00:49, remaining: 2:33:14)
Exported lines 16200-16299 to steamspy_data.csv. Batch 162 time: 0:00:46 (avg: 0:00:49, remaining: 2:32:22)
Exported lines 16300-16399 to steamspy_data.csv. Batch 163 time: 0:00:46 (avg: 0:00:49, remaining: 2:31:31)
Exported lines 16400-16499 t

Exported lines 23100-23199 to steamspy_data.csv. Batch 231 time: 0:00:53 (avg: 0:00:49, remaining: 1:37:28)
Exported lines 23200-23299 to steamspy_data.csv. Batch 232 time: 0:00:54 (avg: 0:00:49, remaining: 1:36:41)
Exported lines 23300-23399 to steamspy_data.csv. Batch 233 time: 0:00:53 (avg: 0:00:49, remaining: 1:35:54)
Exported lines 23400-23499 to steamspy_data.csv. Batch 234 time: 0:00:54 (avg: 0:00:49, remaining: 1:35:07)
Exported lines 23500-23599 to steamspy_data.csv. Batch 235 time: 0:00:54 (avg: 0:00:49, remaining: 1:34:20)
Exported lines 23600-23699 to steamspy_data.csv. Batch 236 time: 0:00:55 (avg: 0:00:49, remaining: 1:33:34)
Exported lines 23700-23799 to steamspy_data.csv. Batch 237 time: 0:00:53 (avg: 0:00:49, remaining: 1:32:46)
Exported lines 23800-23899 to steamspy_data.csv. Batch 238 time: 0:00:53 (avg: 0:00:49, remaining: 1:31:59)
Exported lines 23900-23999 to steamspy_data.csv. Batch 239 time: 0:00:53 (avg: 0:00:49, remaining: 1:31:11)
Exported lines 24000-24099 t

Exported lines 30700-30799 to steamspy_data.csv. Batch 307 time: 0:00:46 (avg: 0:00:49, remaining: 0:35:20)
Exported lines 30800-30899 to steamspy_data.csv. Batch 308 time: 0:00:46 (avg: 0:00:49, remaining: 0:34:31)
Exported lines 30900-30999 to steamspy_data.csv. Batch 309 time: 0:00:47 (avg: 0:00:49, remaining: 0:33:41)
Exported lines 31000-31099 to steamspy_data.csv. Batch 310 time: 0:00:46 (avg: 0:00:49, remaining: 0:32:51)
Exported lines 31100-31199 to steamspy_data.csv. Batch 311 time: 0:00:46 (avg: 0:00:49, remaining: 0:32:02)
Exported lines 31200-31299 to steamspy_data.csv. Batch 312 time: 0:00:47 (avg: 0:00:49, remaining: 0:31:12)
Exported lines 31300-31399 to steamspy_data.csv. Batch 313 time: 0:00:47 (avg: 0:00:49, remaining: 0:30:22)
Exported lines 31400-31499 to steamspy_data.csv. Batch 314 time: 0:00:47 (avg: 0:00:49, remaining: 0:29:33)
Exported lines 31500-31599 to steamspy_data.csv. Batch 315 time: 0:00:47 (avg: 0:00:49, remaining: 0:28:43)
Exported lines 31600-31699 t